<a href="https://colab.research.google.com/github/pjbenard/MPEG/blob/main/JPEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Format de compression MPEG

Notre projet consiste en l'implémentation du format compression MPEG. 
Le projet est divisé en deux parties : la compression JPEG et le "flot optique".

## I. Compression JPEG

La compression JPEG (Joint Photographic Experts Group) est un processus qui permet de réduire la taille d'une image 

Le processus de compression comporte six étapes principales :

1. Transformation de couleurs
2. Sous échantillonnage
3. Découpage en blocs de pixels
4. DCT
5. Quantification
6. Codage RLE et Huffman 


### Importation des librairies

In [ ]:
import numpy as np
import scipy as sp
import scipy.fftpack as fft

#from bokeh.plotting import figure, show
#from bokeh.io import output_notebook
#import holoviews as hv
#hv.config.enable_colab_support = True
#hv.extension('bokeh')

import matplotlib.pyplot as plt

from PIL import Image
import requests
from io import BytesIO

Chargement de l'image. On a le choix entre l'image 'RGB_illumination.jpg' ou 'montagne.jpg'.

In [ ]:

#img_url = "https://upload.wikimedia.org/wikipedia/commons/2/28/RGB_illumination.jpg"
#response = requests.get(img_url)
#img = np.array(Image.open(BytesIO(response.content))).astype(int)

img = np.array(Image.open('images_notebook/montagne.jpg'))

In [ ]:
img.shape

Affichage de l'image et des channels

In [ ]:
def plot_img_channels(img, cmaps=['Reds', 'Greens', 'Blues']):
    fig, axs = plt.subplots(1, 4, figsize=(16, 3), sharey=True, sharex=True)
    axs[0].imshow(img.astype(int))

    for col, cmap in enumerate(cmaps):
        axs[col + 1].imshow(img[...,col], cmap=cmap)

    plt.show()

plot_img_channels(img)

### 1. Transformation des couleurs

D'abort on va transformer les couleurs de l'image.
Les codages de couleur type luminance/chrominance donnent les meilleurs taux de compression car oeil humain assez sensible à la luminosité (luminance) mais peu à la teinte (chrominance) d'une image. (On fera donc un sous échantillonnage de couleurs sur ces couleurs là plutôt que sur RGB)

![Perception](images_notebook/perception.png)

La figure montre que la sensibilité de l'oeil humain est bien différente pour les couleurs rouge, vert et bleue constitutives de nos images. Ainsi le vert est-il le mieux perçu, puis vient le rouge, et enfin le bleu de maniere minoritaire. C'est donc "moins grave" de perdre l'information avec les couleurs type luminance/chrominacne. On passera donc d'une image codée en RGB à une image codée en fonction de sa luminance (Y), et de sa chrominance (Cb, Cr) (format YUV) 



Changer de couleurs RGB à YUV consiste à faire un changement de base orthogonale (Rappel : la base de RGB est orthogonale)

En principe, on a en quelque sorte :

  $$  Y ≃ R + G + B \\
    U ≃ B – Y \\
    V ≃ R – Y$$
    
La matrice de changement de base est plus particulièrement définie ainsi : (Wikipédia + autres sources)

![changement](changement_base.png)

Ici, on utilise simplement une fonction du module Image de la librairie PIL.

Implémentation de RGB_to_YCbCr

In [ ]:
def RGB_to_YCbCr(img_rgb):
    conv = np.array([[ 65.481, 128.553,  24.966], 
                     [-37.797, -74.203, 112.   ], 
                     [112.   , -93.786, -18.214]])
    
    img_ycbcr = np.dot(img_rgb.astype(float)/255, conv.T)
    img_ycbcr[:,:,0] += 16
    img_ycbcr[:,:,[1,2]] += 128
    return img_ycbcr.astype(int)


def YCbCr_to_RGB(img_ycbcr):
    conv = np.array([[1,  0      , 1.402  ], 
                     [1, -0.34414, -.71414], 
                     [1,  1.772  , 0      ]])

    img_rgb = img_ycbcr.astype(float)
    img_rgb[:,:,[1,2]] -= 128
    img_rgb = np.dot(img_rgb, conv.T)
    
    return np.clip(img_rgb, 0, 255).astype(int)

In [ ]:
img_ycbcr = RGB_to_YCbCr(img)
print(img_ycbcr.shape)
plot_img_channels(img_ycbcr, ['gray'] * 3)

In [ ]:
plot_img_channels(YCbCr_to_RGB(img_ycbcr))

Comparaison avec la fonction du module Image de PIL

In [ ]:
img_yuv = Image.open('images_notebook/montagne.jpg').convert('YCbCr')

plot_img_channels(np.array(img_yuv), ['gray'] * 3)

### 2. Sous échantillonnage des couleurs

La deuxième étape de la compression est le sous échantillonnage des couleurs (Cb et Cr). On sépare les channels. On ne touche pas au channel Y mais on va rétrécir les image de U et V. Il y a plusieurs réglages possibles que l'on décrit avec la « notation J:a:b », définie ainsi, par bloc de 8x8 :
- J est le nombre de pixels de Y conservés pour 4 pixels affichés, sur chaque ligne ;
- a est le nombre de pixels de U conservés pour 4 pixels affichés, sur les lignes paires ;
- b est le nombre de pixels de V conservés pour 4 pixels affichés, sur les lignes impaires.
![subsampling](images_notebook/subsampling.png)

Ainsi le sous échantillonnage de couleur le plus utilisé est le 4:2:0 (c'est à dire qu'on découpe l'image en bloc de 8x8). (Mais ce n'est pas le plus important, on peut prendre du 4:4:4)

### 3. Découpage de l'image en blocks


En JPEG, on ne travaille pas sur une image entière : on travaille sur des blocs de 8x8 pixels (séparément en ce qui concerne l’intensité, le bleu et le rouge, donc).
Si la taille d’une image n’est pas exactement un multiple de 8 dans un axe donné, et que
la compression est forte, de légers défauts de compression pourraient apparaître. C’est un des soucis de JPEG.

Chaque bloc de 8x8 est en suite envoyé pour être transformé par DCT.



Translation des coefficients de [0;255] à [-128;127]


In [ ]:
def shift_array(arr, shift=-128):
    return arr + shift

Implémentation de transform_into_blocks

In [ ]:
def transform_into_blocks(img, block_size=8):
    """
    Return a array of size (img.shape[0] // block_size, img.shape[1] // block_size, 3, block_size, block_size) or
                         (3, img.shape[0] // block_size, img.shape[1] // block_size, block_size, block_size) (TBD)
    First shape reads block from top to bottom, from left to right.
    """
    nb_blocks_height = img.shape[0] // block_size
    nb_blocks_width  = img.shape[1] // block_size

    blocks = np.empty((nb_blocks_height, nb_blocks_width, 3, block_size, block_size), dtype=img.dtype)

    for y in range(nb_blocks_height):
        for x in range(nb_blocks_width):
            for color in range(3):
                blocks[y, x, color] = img[y * block_size:(y + 1) * block_size, 
                                          x * block_size:(x + 1) * block_size, 
                                          color]

    return blocks

In [ ]:
blocks = transform_into_blocks(img_ycbcr)

In [ ]:
blocks.shape

In [ ]:
b1 = shift_array(blocks[0, 0, 1])
b1

Implémentation de transform_into_image (opération inverse)

In [ ]:
def transform_into_image(blocks, block_size=8):
    """
    Return a array of size (img.shape[0] // block_size, img.shape[1] // block_size, 3, block_size, block_size) or
                         (3, img.shape[0] // block_size, img.shape[1] // block_size, block_size, block_size) (TBD)
    First shape reads block from top to bottom, from left to right.
    """
    img_height = blocks.shape[0] * block_size
    img_width  = blocks.shape[1] * block_size

    img = np.empty((img_height, img_width, 3), dtype=blocks.dtype)

    for i in range(blocks.shape[0]):
        for j in range(blocks.shape[1]):
            for color in range(3):
                img[
                    i * block_size : (i + 1) * block_size, 
                    j * block_size : (j + 1) * block_size, 
                    color,
                ] = blocks[i, j, color]

    return img

In [ ]:
blocks.shape

In [ ]:
img_deblocked = transform_into_image(blocks)
img_deblocked.shape

In [ ]:
plot_img_channels(img_deblocked, ['gray'] * 3)

### 4. Transformée en cosinus discret 

On fait une transformée DCT soit Discrete Cosine Transform. On applique cette transfo numérique à chaque bloc (variante de la transfo de fourier). Cette transfo décompose un bloc (considéré comme une fc num à deux variables) en une somme de fc cosinus oscillant à des freq différentes. Chaque bloc est ainsi décrit en une carte de freq et en amplitude plutôt qu'en pixels et coeff de couleur. (formule de la DCT dispo sur wiki) 

Le calcul d'une DCT est l'étape qui coûte le plus de temps et de ressources dans la compression JPEG. Mais elle peremt de séparer les basses et hautes freq de l'image. 

In [ ]:
dct1 = fft.dctn(b1)
dct1.shape, dct1.astype(int)

In [ ]:
dct1 = fft.dctn(b1, norm='ortho')
dct1.shape, dct1.astype(int)

Implémentation DCT

In [ ]:
def apply_dct(blocks):
    return fft.dctn(blocks, axes=[-2, -1], norm ='ortho')

In [ ]:
blocks_dct = apply_dct(shift_array(blocks))
blocks_dct.shape

Implémentation DCT inverse

In [ ]:
def apply_idct(blocks):
    return fft.idctn(blocks, axes=[-2,-1], norm='ortho')

In [ ]:
blocks_idct = apply_idct(blocks_dct)
print(blocks_idct.shape)
print(blocks_idct[0, 0, 1])


In [ ]:
#blocks = np.random.randn(16, 16)
np.allclose(blocks, apply_idct(apply_dct(blocks)))


### 5. Quantification

C'est à cette étape que l'on perd l'information

In [ ]:
quantization_matrix = np.array([[16, 11, 10, 16,  24,  40,  51,  61],
                                [12, 12, 14, 19,  26,  58,  60,  55],
                                [14, 13, 16, 24,  40,  57,  69,  56],
                                [14, 17, 22, 29,  51,  87,  80,  62], 
                                [18, 22, 37, 56,  68, 109, 103,  77], 
                                [24, 35, 55, 64,  81, 104, 113,  92], 
                                [49, 64, 78, 87, 103, 121, 120, 101], 
                                [72, 92, 95, 98, 112, 100, 103,  99]], dtype=int)

In [ ]:
def quantize(arr, quant_mat=quantization_matrix):
    return np.round(np.divide(arr, quant_mat)).astype(int)

def dequantize(arr, quant_mat=quantization_matrix):
    return np.multiply(arr, quant_mat)

In [ ]:
blocks_quant = quantize(blocks_dct)

In [ ]:
blocks_quant[0, 0, 1]

In [ ]:
blocks_dequant = dequantize(blocks_quant)

In [ ]:
blocks.shape

In [ ]:
img_deblocked = transform_into_image(apply_idct(blocks_dct))
img_deblocked.shape

In [ ]:
plot_img_channels(YCbCr_to_RGB(shift_array(img_deblocked, 128)))

### 6. Codage RLE et Huffman 



http://www-ljk.imag.fr/membres/Valerie.Perrier/SiteWeb/node6.html